In [1]:
import os

import pandas as pd

os.path.abspath(os.curdir)

'/Users/nicholsn/Repos/metasearch/crawler/load'

In [2]:
# Load all the transformed files into a dataframes to be merged.
load_path = os.path.abspath(os.curdir)
xfm_path = os.path.abspath('../transform')
project_names = [project for project in os.listdir(xfm_path) if '.' not in project]
csv_files = list()
for project_name in project_names:
    xfm_dir = os.path.join(xfm_path, project_name)
    files = os.listdir(xfm_dir)
    [csv_files.append(os.path.join(xfm_path, project_name, csv)) for csv in files if '.csv' in csv]

In [3]:
# Define the order of the columns we want in our final table.
columns = ['project',
           'site_id',
           'participant_id',
           'diagnosis',
           'sex', 
           'age',
           'handedness',
           'full iq',
           'performance iq',
           'verbal iq']

In [35]:
df_list = [pd.read_csv(i) for i in csv_files]
df = pd.concat(df_list)
df.to_csv(os.path.join(load_path, 'phenotype.csv'), columns=columns, index=False)
df.head()

,age,diagnosis,dsm_iv_tr,full iq,handedness,participant_id,performance iq,project,sex,site_id,verbal iq
0,16.77,autism,autism,103.0,ambidextrous,50002,89.0,abide_initiative,Male,PITT,116.0
1,24.45,autism,autism,124.0,right handed,50003,115.0,abide_initiative,Male,PITT,128.0
2,19.09,autism,autism,113.0,right handed,50004,117.0,abide_initiative,Male,PITT,108.0
3,13.73,autism,autism,119.0,right handed,50005,118.0,abide_initiative,Female,PITT,117.0
4,13.37,autism,autism,109.0,left handed,50006,119.0,abide_initiative,Male,PITT,99.0


In [5]:
img_path = os.path.abspath( '../fcp-indi-new.gz')
img = pd.read_csv(img_path, header=None)
df = img[0]

In [6]:
paths = df.apply(lambda x: x.split(' ')[-1])

In [7]:
bucket = 'https://s3.amazonaws.com/fcp-indi/'
aws_paths = paths.apply(lambda x: ''.join([bucket, x]))

In [8]:
# Break the path into parts.
lists = aws_paths.str.split('/')

In [9]:
# Pull the project name out of the path.
def get_project(x):
    try:
        return x[6].lower()
    except:
        return None
projects = lists.apply(get_project)

In [10]:
final = pd.DataFrame(data=dict(project=projects, url=aws_paths))

In [11]:
nii = final[final.url.str.endswith('nii.gz')]

In [13]:
abide= nii[nii.project == 'abide_initiative']

In [19]:
abide_raw = abide[abide.url.str.contains('RawData')]

In [22]:
abide_anat = abide_raw[abide_raw.url.str.contains('anat')]

In [27]:
abide_list = abide_anat.url.str.split('/')

In [31]:
def get_abide_sub(x):
    try:
        return x[9]
    except:
        return None
abide_sub = abide_list.apply(get_abide_sub)

In [43]:
abide_anat['participant_id'] = abide_sub.apply(lambda x: x[2:])

/Users/nicholsn/miniconda/envs/metasearch/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [104]:
df.participant_id = df.participant_id.astype(pd.np.str)
pheno = df.set_index(['participant_id'], drop=False)

In [105]:
anat = abide_anat.set_index(['participant_id'], drop=False)

In [106]:
pheno['t1_url'] = anat.url

In [74]:
r = pd.merge(pheno, anat, how='left', right_on='participant_id', left_on='participant_id')

In [107]:
pheno

,age,diagnosis,dsm_iv_tr,full iq,handedness,participant_id,performance iq,project,sex,site_id,verbal iq,t1_url
participant_id,,,,,,,,,,,,
50002,16.77,autism,autism,103.0,ambidextrous,50002,89.0,abide_initiative,Male,PITT,116.0,https://s3.amazonaws.com/fcp-indi/data/Project...
50003,24.45,autism,autism,124.0,right handed,50003,115.0,abide_initiative,Male,PITT,128.0,https://s3.amazonaws.com/fcp-indi/data/Project...
50004,19.09,autism,autism,113.0,right handed,50004,117.0,abide_initiative,Male,PITT,108.0,https://s3.amazonaws.com/fcp-indi/data/Project...
50005,13.73,autism,autism,119.0,right handed,50005,118.0,abide_initiative,Female,PITT,117.0,https://s3.amazonaws.com/fcp-indi/data/Project...
50006,13.37,autism,autism,109.0,left handed,50006,119.0,abide_initiative,Male,PITT,99.0,https://s3.amazonaws.com/fcp-indi/data/Project...
50007,17.78,autism,autism,110.0,right handed,50007,112.0,abide_initiative,Male,PITT,106.0,https://s3.amazonaws.com/fcp-indi/data/Project...
50008,32.45,autism,autism,123.0,right handed,50008,114.0,abide_initiative,Male,PITT,123.0,https://s3.amazonaws.com/fcp-indi/data/Project...
50009,33.86,autism,autism,126.0,right handed,50009,128.0,abide_initiative,Male,PITT,118.0,https://s3.amazonaws.com/fcp-indi/data/Project...
50010,35.2,autism,autism,81.0,left handed,50010,93.0,abide_initiative,Male,PITT,81.0,https://s3.amazonaws.com/fcp-indi/data/Project...


In [108]:
anat

,project,url,participant_id
participant_id,,,
50642,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...,50642
50646,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...,50646
50647,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...,50647
50649,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...,50649
50653,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...,50653
50654,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...,50654
50656,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...,50656
50659,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...,50659
50660,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...,50660


In [100]:
pheno.loc[50142]

age                          13.99
diagnosis                   autism
dsm_iv_tr                      NaN
full iq                        124
handedness             left handed
participant_id               50142
performance iq                 NaN
project           abide_initiative
sex                           Male
site_id                       OHSU
verbal iq                      NaN
t1_url                         NaN
Name: 50142, dtype: object

In [109]:
final

,project,url
0,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...
1,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...
2,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...
3,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...
4,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...
5,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...
6,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...
7,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...
8,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...
9,abide_initiative,https://s3.amazonaws.com/fcp-indi/data/Project...
